In [1]:
!pip install pandas nltk scikit-learn openpyxl

In [2]:
import pandas as pd
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to C:\Users\Jason San
[nltk_data]     uWu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Jason San
[nltk_data]     uWu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
def load_data(path, sheet_name):
    xls_file = pd.ExcelFile(path)
    dataframe = pd.read_excel(xls_file, sheet_name)
    dataframe = dataframe[['Question', 'Answer']]
    dataframe = dataframe.dropna().reset_index(drop=True)
    return dataframe

df = load_data("FAQs.xlsx", "Catalogs")
df.head()


,Question,Answer
0,How to create a sales order in SAP?,"To create a sales order in SAP, use transactio..."
1,How to generate a vendor invoice in SAP?,"To generate a vendor invoice in SAP, use trans..."
2,How to post GL in SAP?,"To post a GL entry in SAP, use transaction cod..."
3,How to create a purchase order in SAP?,"To create a purchase order in SAP, use transac..."
4,How to run a material availability check in SAP?,"To run a material availability check in SAP, u..."


In [4]:
def clean_text(text):
    word_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
    tokens = word_tokenizer.tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(word.lower().translate(remove_punct_dict)) for word in tokens]
    # print(f"Original text: {text}")
    # print(f"Tokenized: {tokens}")
    # print(f"Lemmatized: {lemmatized_tokens}")
    return lemmatized_tokens


In [6]:
def process_question(user_question, dataframe):
    questions = dataframe["Question"].tolist()
    questions.append(user_question)
    TfidfVec = TfidfVectorizer(tokenizer=clean_text, stop_words='english')
    tfidf = TfidfVec.fit_transform(questions)
    
    print("\nTF-IDF Matrix:")
    # print(tfidf.toarray())
 
    
    vals = cosine_similarity(tfidf[-1], tfidf)
    print("\nCosine Similarity Values:")
    print(vals)
    
    
    idx = vals.argsort()[0][-2]
    print("\nIndex of closest question:", idx)
    
    flat = vals.flatten()
    flat.sort()
    # print()
    req_tfidf = flat[-2]

    del questions[-1]

    if req_tfidf == 0:
        return "Sorry, I didn't get you"
    else:
        return dataframe.Answer[idx]


In [7]:
user_question = "invoice procedure?"
answer = process_question(user_question, df)
print("Question:", user_question)
print("Answer:", answer)


c:\SAP BTP AI\GenAI\git\gen-ai-3\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(



TF-IDF Matrix:

Cosine Similarity Values:
[[0.         0.29266741 0.         0.         0.         0.
  0.         0.4727373  0.         0.         0.         0.
  0.         0.         0.         0.         1.        ]]

Index of closest question: 7
Question: invoice procedure?
Answer: To create an invoice in SAP, use transaction code VF01. Enter the billing document and process the invoice.


c:\SAP BTP AI\GenAI\git\gen-ai-3\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'u', 'wa'] not in stop_words.
  warnings.warn(
